In [1]:
import pandas as pd
import numpy as np

from warehouse_modeling.induced_backorder_cost import *
from warehouse_modeling.lead_time_approximation import *
from warehouse_modeling.warehouse_optimization import *
from warehouse_modeling.warehouse_demand_modeling import *

from single_echelon_utils.inventory_level_computation import *
from single_echelon_utils.service_level_computation import *
from single_echelon_utils.dealer_optimization import *

from utils import *

## INDATA
First, read indata from a specified excel file and sheet.

In [2]:
excel_path = "/Volumes/GoogleDrive/.shortcut-targets-by-id/10oYqI9u7nCLK0q7xF2CvGGIQVokusjaI/Exjobb/9. Analytical modeling/test_indata.xlsx"
indata_sheet = "test_case_1_whNormal_NBD"
indataDF = pd.read_excel(excel_path,indata_sheet)
outdataDF = indataDF.copy()


In [3]:
Q_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Q").to_numpy()
mu_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand mean").to_numpy()
demand_type_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand type").to_numpy()

for id in indataDF.get(indataDF["Type"] == "Dealer").get("Installation id"):
    if indataDF.get(indataDF["Installation id"] == id).get("Demand type").to_numpy() == "Poisson":
        indataDF.loc[indataDF["Installation id"] == id, "Demand stdev"] = math.sqrt(indataDF.loc[outdataDF["Installation id"]== id, "Demand mean"].to_numpy())

# New change

sigma_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Demand stdev").to_numpy()


Q_subbatch_size = find_smallest_divisor(Q_dealer_arr)
L_wh = float(indataDF.get(indataDF["Type"]=="RDC").get("Transport time"))

In [4]:

rdc_f_u_probability_array, wh_dist, mu_L, sigma2_L = warehouse_subbatch_demand_probability_array(Q_dealer_arr, mu_dealer_arr, 
    sigma_dealer_arr, demand_type_arr, L_wh, Q_subbatch_size)


outdataDF.loc[outdataDF["Type"] == "Dealer","Q, subbatches"] = Q_dealer_arr/Q_subbatch_size

outdataDF.loc[outdataDF["Type"] == "RDC","Demand type"] = wh_dist
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand mean"] = mu_L * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Lead time demand stdev"] = math.sqrt(sigma2_L) * Q_subbatch_size
outdataDF.loc[outdataDF["Type"] == "RDC","Demand mean"] = mu_L * Q_subbatch_size/L_wh
outdataDF.loc[outdataDF["Type"] == "RDC","Demand stdev"] = math.sqrt(sigma2_L * Q_subbatch_size/L_wh)
outdataDF.loc[outdataDF["Type"] == "RDC", "Demand variance"] = sigma2_L * Q_subbatch_size/L_wh



RDC reorder-point optimization

In [5]:
h_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Holding cost").to_numpy()
fill_rate_target_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Target item fill rate").to_numpy()
p_dealer_arr = fill_rate_target_arr*h_dealer_arr/(np.ones_like(fill_rate_target_arr)-fill_rate_target_arr)
l_dealer_arr = indataDF.get(indataDF["Type"] == "Dealer").get("Transport time").to_numpy()
mu_wh = mu_L/L_wh * Q_subbatch_size

beta_list = []
for h,Q,p,l,my,sigma in zip(h_dealer_arr,Q_dealer_arr,p_dealer_arr,l_dealer_arr,mu_dealer_arr,sigma_dealer_arr):
    beta_list.append(induced_backorder_cost_opt(h,Q,p,l,my,sigma))

beta_arr = np.array(beta_list)

beta_rdc = weighting_backorder_cost(mu_dealer_arr,mu_wh,beta_arr)
outdataDF.loc[outdataDF["Type"] == "Dealer", "Shortage cost"] = p_dealer_arr
outdataDF.loc[outdataDF["Type"] == "RDC", "Beta"] = beta_rdc
outdataDF.loc[outdataDF["Type"] == "Dealer", "Beta"] = beta_arr
#print(f"Optimal weighted induced backorder cost at the warehouse is: {beta_rdc}, betas are: {beta_arr}")



In [6]:
h_rdc = float(indataDF.get(indataDF["Type"] == "RDC").get("Holding cost"))
Q_0 = int(int(indataDF.get(indataDF["Type"] == "RDC").get("Q"))/Q_subbatch_size)

R_0 = warehouse_optimization(Q_subbatch_size,Q_0,rdc_f_u_probability_array,h_rdc,beta_rdc)

outdataDF.loc[outdataDF["Type"] == "RDC", "Q, subbatches"] = Q_0
outdataDF.loc[outdataDF["Type"] == "RDC", "R, subbatches"] = R_0
#outdataDF


Starting optimizing, R = 0, c = 91.3552749736296, c+1 = 87.10622171528978
Doing upwards optimizing, R = 1, c = 87.10622171528978, c+1 = 82.85722161197488
Doing upwards optimizing, R = 2, c = 82.85722161197488, c+1 = 78.60835479386793
Doing upwards optimizing, R = 3, c = 78.60835479386793, c+1 = 74.35980635768891
Doing upwards optimizing, R = 4, c = 74.35980635768891, c+1 = 70.11198257512515
Doing upwards optimizing, R = 5, c = 70.11198257512515, c+1 = 65.8657306834497
Doing upwards optimizing, R = 6, c = 65.8657306834497, c+1 = 61.622729112809196
Doing upwards optimizing, R = 7, c = 61.622729112809196, c+1 = 57.386135977347465
Doing upwards optimizing, R = 8, c = 57.386135977347465, c+1 = 53.161593683541874
Doing upwards optimizing, R = 9, c = 53.161593683541874, c+1 = 48.9586709910728
Doing upwards optimizing, R = 10, c = 48.9586709910728, c+1 = 44.792763066063586
Doing upwards optimizing, R = 11, c = 44.792763066063586, c+1 = 40.687352632870855
Doing upwards optimizing, R = 12, c = 4

In [7]:
W = waiting_time(negative_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array),L_wh,mu_L,Q_subbatch_size)
outdataDF.loc[outdataDF["Type"]== "Dealer", "Waiting time"] = W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time"] = outdataDF.get(outdataDF["Type"]== "Dealer").get("Transport time").to_numpy() + W
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand mean"] = outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand mean").to_numpy()
outdataDF.loc[outdataDF["Type"] == "Dealer", "Lead time demand stdev"] =outdataDF.get(outdataDF["Type"]== "Dealer").get("Lead time").to_numpy()*outdataDF.get(outdataDF["Type"]== "Dealer").get("Demand stdev").to_numpy()
outdataDF.loc[outdataDF["Type"] == "RDC", "Lead time"] = indataDF.get(indataDF["Type"] == "RDC").get("Transport time")

In [8]:
opt_dealer_list = []
L_dealer_arr = outdataDF.get(outdataDF["Type"] == "Dealer").get("Lead time")
for Q,L_est,fill_rate_target,demand_type,mu,sigma in zip(Q_dealer_arr,L_dealer_arr,fill_rate_target_arr,demand_type_arr, mu_dealer_arr,sigma_dealer_arr):
    print(demand_type, mu, sigma)
    opt_dealer_list.append(dealer_R_optimization(Q,L_est,fill_rate_target,demand_type,mu,demand_variance = math.pow(sigma,2)))

R_opt_dealer_list,fill_rate_dealer_list,exp_stock_on_hand_list = [],[],[]
for tup in opt_dealer_list:
    R_opt_dealer_list.append(tup[0])
    fill_rate_dealer_list.append(tup[2])
    exp_stock_on_hand_list.append(tup[3])

outdataDF.loc[outdataDF["Type"] == "Dealer", "R optimal"] = R_opt_dealer_list
outdataDF.loc[outdataDF["Type"] == "Dealer", "Realized item fill rate"] = fill_rate_dealer_list
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected stock on hand"] = exp_stock_on_hand_list

outdataDF.loc[outdataDF["Type"] == "RDC","R optimal"] = R_0*Q_subbatch_size # In units.
outdataDF.loc[outdataDF["Type"] == "RDC","Expected stock on hand"] = positive_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array)
outdataDF.loc[outdataDF["Type"] == "RDC","Expected backorders"] = negative_inventory(Q_subbatch_size,Q_0,R_0,rdc_f_u_probability_array)


NBD 1.0 1.1
NBD 1.0 1.2
NBD 1.0 1.3
NBD 1.0 1.4
NBD 1.0 1.5
NBD 1.0 1.6
NBD 1.0 1.7
NBD 1.0 1.8
NBD 1.0 1.9
NBD 1.0 2.0


In [9]:
# List comprehension for exp backorders.
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected backorders"] = [ 
    expected_backorders_discrete(R,Q,lt_mu,exp_stock_on_hand) for 
    R,Q,lt_mu,exp_stock_on_hand in zip(R_opt_dealer_list,Q_dealer_arr,
    outdataDF.get(outdataDF["Type"] == "Dealer").get("Lead time demand mean").to_numpy(),
    exp_stock_on_hand_list) ]

In [10]:
# Adding costs
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected holding costs per time unit"] = h_dealer_arr*np.array(exp_stock_on_hand_list)
outdataDF.loc[outdataDF["Type"] == "Dealer", "Expected shortage costs per time unit"] = outdataDF.get(
    outdataDF["Type"] == "Dealer").get("Expected backorders").to_numpy()*p_dealer_arr
outdataDF.loc[outdataDF["Type"] == "Dealer", "Total expected costs"] = outdataDF.get(outdataDF["Type"] == "Dealer").get("Expected holding costs per time unit").to_numpy() + outdataDF.get(outdataDF["Type"] == "Dealer").get("Expected shortage costs per time unit").to_numpy()

outdataDF.loc[outdataDF["Type"] == "RDC", "Expected holding costs per time unit"] = h_rdc*outdataDF.get(outdataDF["Type"] == "RDC").get("Expected stock on hand").to_numpy()
outdataDF.loc[outdataDF["Type"] == "RDC", "Total expected costs"] = outdataDF.get(outdataDF["Type"] == "RDC").get("Expected holding costs per time unit").to_numpy()



In [11]:
#excel_path_outdata = "/Users/AlexanderLarsson/documents/VSCode/outdata.xlsx"
#outdataDF.to_excel(excel_path_outdata,sheet_name = "Outdata_latest_testrun")

outdataDF


,Installation id,Type,Name,Transport time,Q,Holding cost,Target item fill rate,Demand type,Demand mean,Demand stdev,...,"R, subbatches",Waiting time,Lead time,R optimal,Realized item fill rate,Expected stock on hand,Expected backorders,Expected holding costs per time unit,Expected shortage costs per time unit,Total expected costs
0,1,RDC,Johannesburg,10,24,1,NaN,Normal,10.0,2.800459,...,22.0,NaN,10.000000,88.0,NaN,8.608495,6.608442,8.608495,NaN,8.608495
1,2,Dealer,Deal1,10,4,1,0.95,NBD,1.0,1.100000,...,NaN,0.660844,10.660844,16.0,0.963919,7.885958,0.046802,7.885958,0.889233,8.775191
2,3,Dealer,Deal2,3,8,1,0.95,NBD,1.0,1.200000,...,NaN,0.660844,3.660844,6.0,0.963273,6.871569,0.032413,6.871569,0.615853,7.487423
3,4,Dealer,Deal3,2,4,1,0.95,NBD,1.0,1.300000,...,NaN,0.660844,2.660844,6.0,0.961531,5.868688,0.029532,5.868688,0.561103,6.429790
4,5,Dealer,Deal4,1,8,1,0.95,NBD,1.0,1.400000,...,NaN,0.660844,1.660844,4.0,0.965817,6.862288,0.023132,6.862288,0.439508,7.301796
5,6,Dealer,Deal5,4,4,1,0.95,NBD,1.0,1.500000,...,NaN,0.660844,4.660844,10.0,0.951184,7.901209,0.062053,7.901209,1.179002,9.080210
6,7,Dealer,Deal6,3,8,1,0.95,NBD,1.0,1.600000,...,NaN,0.660844,3.660844,8.0,0.955490,8.894026,0.054870,8.894026,1.042529,9.936555
7,8,Dealer,Deal7,2,4,1,0.95,NBD,1.0,1.700000,...,NaN,0.660844,2.660844,8.0,0.950016,7.893438,0.054283,7.893438,1.031371,8.924809
8,9,Dealer,Deal8,5,8,1,0.95,NBD,1.0,1.800000,...,NaN,0.660844,5.660844,13.0,0.959102,11.910078,0.070922,11.910078,1.347514,13.257591
9,10,Dealer,Deal9,4,4,1,0.95,NBD,1.0,1.900000,...,NaN,0.660844,4.660844,13.0,0.950772,10.920950,0.081794,10.920950,1.554091,12.475041


In [12]:
print(f"Length: {len(rdc_f_u_probability_array)}, p(0) = {rdc_f_u_probability_array[0]}")

Length: 47, p(0) = 1.5732901120446164e-08


In [13]:
outdataDF.get("Lead time demand stdev")

0     17.711658
1     11.726929
2      4.393013
3      3.459097
4      2.325182
5      6.991266
6      5.857351
7      4.523435
8     10.189520
9      8.855604
10     7.321688
Name: Lead time demand stdev, dtype: float64